In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature = 0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/1984_chap_one.txt")

doc = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(doc, cached_embeddings)

retriever = vectorstore.as_retriever()

# list of docs

# for doc in list of docs | prompt | llm

# for respone in list of llms response | put them all together

# final doc | prompt | llm

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

# def map_docs(inputs):
#     print(inputs)
#     documents = inputs["documents"]
#     question = inputs["question"]
#     results = []
#     for document in documents:
#         result = map_doc_chain.invoke(
#             {"context": document.page_content, "question": question}
#         ).content
#         results.append(result)
#     print(results)
#     return "hello"

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
        {"context": doc.page_content, "question": question}
        ).content 
        for doc in documents
    )

map_chain = {
    "documents": retriever, 
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
     Given the following extracted parts of a long document and a 
     question, create a final answer. 
     If you don't know the answer, just say that you don't know. Don't 
     try to make up an answer.
     ------
     {context}
     """,),
     ("human", "{question}"),

])

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

# chain.invoke("Describe Victory Mansions")
chain.invoke("Where does Winston go to work?")


/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/ipykernel_5044/2872659247.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/ipykernel_5044/2872659247.py:22: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  loader = UnstructuredFileLoader("./files/1984_chap_one.txt")
/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/ipykernel_5044/2872659247.py:

RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-DxyR7ijEF9ww6YcVJ24D6t40 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.